# Draw

By going through the pots from Pot A to Pot D.

1.   Choose a team randomly
2.   Successively assign d matches from each pot to it, each time checking if a pairing exists

In [ ]:
def is_pickable_one_pot(G, e, d_v, d):
    """Returns True if there exists a d-matching when e is added to the matching"""
    u, v = e[0], e[1]
    nb_edges  = len(list(G.edges()))
    nb_nodes = len(list(G.nodes()))
    if nb_nodes == 2 and nb_edges == 1:
      return True
    if nb_nodes == d+1 and nb_edges == d+1:
        return (d_v[u] == d or d_v[v] == d)

    d_v_copy = d_v.copy()
    # Remove a neighbor from the nodes of the chosen edge
    d_v_copy[u] -= 1
    d_v_copy[v] -= 1
    G_actualized = G.copy()
    G_actualized.remove_edge(u, v)
    # Remove nodes that already have d-neighbors
    if d_v_copy[u] == 0:
        G_actualized.remove_node(u)
    if d_v_copy[v] == 0:
        G_actualized.remove_node(v)
    """
    print("Actualized graph : ", G_actualized)
    print("d_v actualized : ", d_v_copy)

    plt.figure()
    pos = nx.circular_layout(G_actualized)
    nx.draw(G_actualized, pos,with_labels = True, **options)
    """
    G_ = transformed_graph(G_actualized, d_v_copy, show=False)

    # Edmonds' algorithm on graph G
    graph = Graph()
    graph.nodes = list(G_.nodes())
    edges_ = []
    for e in G_.edges():
        u, v = e[0], e[1]
        edges_.append([u,v])
    graph.edges = edges_
    matching = Matching()
    matching = find_maximum_matching(graph, matching).edges

    return test_adapted(G_actualized, d_v_copy, matching)

# Initialisation du nombre de voisins à d

G_single_pot_adapted_test = single_pot_graph(pots[0], show=False)
G_single_pot_adapted_test.remove_edge("FC Barcelona", "Manchester United")
d_v_init = dict()
for node in G_single_pot_adapted_test.nodes():
  d_v_init[node] = 2

d_v_init['FC Barcelona'] = 1
d_v_init['Manchester United'] = 1


e = ("Juventus", "Paris Saint-Germain")

is_pickable_one_pot(G_single_pot_adapted_test, e, d_v_init,d=3)


In [ ]:
def draw_one_pot(pot, d=2, show = False):
    """
    Draw a perfect d-matching in the pot.

    Args:
    - G: networkx graph
    - d_v: dictionary mapping nodes to their degree in the d-matching
    - d (int)

    Returns:
    - A matching
    """
    # Creating the graph for a pot
    G = single_pot_graph(pot)

    # Initialisation du nombre de voisins à d
    d_v = dict()
    for node in G.nodes():
      d_v[node] = d

    G_actualized = G.copy()
    d_v_copy = d_v.copy()
    matching = []
    nb_nodes = len(list(G.nodes()))

    while len(matching) != (d*nb_nodes)//2:
        # Pick a random edge
        edges = list(G_actualized.edges())
        e = edges[random.randint(0, len(edges) - 1)]
        # Check if the edge can be added to the matching
        if is_pickable_one_pot(G_actualized, e, d_v_copy,d):
          # Update the matching
          matching.append(e)
          # Update the actualized graph and the d_v dictionary
          u, v = e
          d_v_copy[u] -= 1
          d_v_copy[v] -= 1
          G_actualized.remove_edge(u, v)
          if d_v_copy[u] == 0:
              G_actualized.remove_node(u)
          if d_v_copy[v] == 0:
              G_actualized.remove_node(v)
          if show:
            plt.figure()
            pos = nx.circular_layout(G)
            nx.draw(G_actualized,pos, with_labels = True, **options)
            plt.show()
          #print(f'The match picked is : {e}')
    print('\n')
    print("Here's a calendar of games : ", "\n")
    for game in matching:
      print(game[0], ' vs ' , game[1])
    print('\n')

for pot in pots:
  draw_one_pot(pot)

In [ ]:
def is_pickable_2_pots(G, e, d_v, show = False):
    """
    Returns True if a subset F has been found such that
    ∀v ∈ V degF (v) = d in the bipartite graph between Pot_i and Pot_j where i!=j
    """
    u, v = e[0], e[1]
    nb_edges  = len(list(G.edges()))
    nb_nodes = len(list(G.nodes()))
    if nb_nodes == 2:
      return nb_edges == 1
    d_v_copy = d_v.copy()
    # Remove a neighbor from the nodes of the chosen edge
    d_v_copy[str(u)] -= 1
    d_v_copy[str(v)] -= 1
    G_actualized = G.copy()
    G_actualized.remove_edge(u, v)
    # Remove nodes that already have d-neighbors
    if d_v_copy[u] == 0:
        G_actualized.remove_node(u)
    if d_v_copy[v] == 0:
        G_actualized.remove_node(v)

    if not nx.is_connected(G_actualized):
      return False


    # Create maximum flow graph
    G_flow = max_flow_graph_adapted(G_actualized, d_v_copy)

    # Find maximum flow with Edmonds-Karp algorithm
    flow_value, flow_dict = nx.maximum_flow(G_flow, 's', 't', flow_func=edmonds_karp)

    # Position of edges
    # Get the nodes and the edges of the bipartite graph
    bottom_nodes, top_nodes = bipartite.sets(G_actualized)
    U = list(bottom_nodes)
    V = list(top_nodes)
    E = list(G_actualized.edges())

    pos = dict()
    pos['s'] = (0,6)
    pos['t'] = (18,6)
    for i, node in enumerate(U):
        pos[node] = (6, i + 2)
    for i, node in enumerate(V):
        pos[node] = (12, i + 2)

    # Display graph
    if show:
        display(G_actualized, pos, "s-t maximum flow graph")


    # Check if all nodes in V have a flow degree greater than or equal to d
    result = True
    for v in V:
        flow = 0
        for u in flow_dict[v]:
            flow += flow_dict[v][u]
        result = (flow == d_v_copy[v])
    for u in U:
        flow = 0
        for v in flow_dict[u]:
            flow += flow_dict[u][v]
        result = (flow == d_v_copy[u])

    if show:
        # Display result
        if result:
            # Build matching by selecting charged edges
            matching = []
            for u,v in G_actualized.edges():
                if flow_dict[u][v] == 1:
                    matching.append((u,v))
            print(f"A subset F has been found such that ∀v ∈ V degF (v).")
            # Display matching
            print("Resulting graph with charged edges")
            print("\n")
            print("=================================================================")
            print("\n")
            print(matching)
            print("\n")
            print("===================================================================")
            plt.figure(figsize = (30,30))
            display_matching_in_bipartite_graph(U, V, flow_dict, "Matching found")
        else:
            print(f"No subset F was found such that ∀v ∈ V degF (v).")

    return result

# Initialisation du nombre de voisins à d
d_v_init = dict()
G_A_B = initialize_graph(pots[0], pots[1])
for node in G_A_B.nodes():
  d_v_init[node] = 2
G_A_B.remove_edge("Liverpool", "Villarreal")
d_v_init["Liverpool"] = 1
d_v_init["Villarreal"] = 1

e = ("Liverpool", "Benfica")
is_pickable_2_pots(G_A_B, e, d_v_init, show = True)


In [ ]:
def draw_two_pots(Pot_i, Pot_j, d=2, show = False):
    """
    Realise a draw between two distinct pots

    Args:
    - G: networkx graph
    - d_v: dictionary mapping nodes to their degree in the d-matching

    Returns:
    - A matching
    """
    # Creating bipartite graph
    G = initialize_graph_adapted(Pot_i, Pot_j)

    # Initialisation du nombre de voisins à d
    d_v = dict()
    for node in G.nodes():
      d_v[node] = d

    G_actualized = G.copy()
    d_v_copy = d_v.copy()
    matching = []
    nb_nodes = len(list(G.nodes()))
    while len(matching) < (d*nb_nodes)//2:
        # Pick a random edge
        edges = list(G_actualized.edges())
        e = edges[random.randint(0, len(edges) - 1)]
        if len(edges) == 1:
          matching.append(e)
          break
        # Check if the edge can be added to the matching
        if is_pickable_2_pots(G_actualized, e, d_v_copy):
          # Update the matching
          matching.append(e)
          # Update the actualized graph and the d_v dictionary
          u, v = e
          d_v_copy[u] -= 1
          d_v_copy[v] -= 1
          G_actualized.remove_edge(u, v)
          if d_v_copy[u] == 0:
              G_actualized.remove_node(u)
          if d_v_copy[v] == 0:
              G_actualized.remove_node(v)
          #print(f'The match picked is : {e}')
          if show:
            #print(G_actualized)
            #print("d_v : ", d_v_copy)
            #print("match try : ", e)
            bottom_nodes, top_nodes = bipartite.sets(G_actualized)
            U = list(bottom_nodes)
            V = list(top_nodes)
            E = list(G_actualized.edges())
            pos = dict()
            pos['s'] = (0,6)
            pos['t'] = (18,6)
            for i, node in enumerate(U):
                pos[node] = (6, i + 2)
            for i, node in enumerate(V):
                pos[node] = (12, i + 2)
            # Display graph
            display(G_actualized, pos,"draw")
          #print(f'The match try picked is : {e}')
    print("\n","Here's a calendar of games : ", "\n")
    for game in matching:
      print(game[0], ' vs ' , game[1])

draw_two_pots(pots[0], pots[1],show = True)

"""
for i in range(10):
  for i, pot in enumerate(pots):
    for j in range(i+1, len(pots)):
        draw_two_pots(pot, pots[j],show = True)
"""

In [ ]:
def draw(pots):
  for i, pot in enumerate(pots):
    for j in range(i+1, len(pots)):
        draw_two_pots(pot, pots[j])

  for pot in pots:
    draw_one_pot(pot)

for i in range(50):
  draw(pots)